In [1]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, random
import yaml

In [2]:
million=1000000
nbMillions=200
nTiles=1
lexiquePrefix="MGC-160104"
tiragePrefix="MGC-160721-0"
echantillon="-%dMo"%nbMillions

In [3]:
with open("/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+lexiquePrefix+'-Verbes2.pkl', 'rb') as input:
    lexique = pickle.load(input)

In [4]:
lexique["freqcum"]= (lexique["freq"].cumsum()*1000).astype(int)

In [5]:
rangeTop=lexique["freqcum"].max()

In [6]:
del lexique["ext"]
del lexique["cs"]
del lexique["ms"]
del lexique["vs"]
del lexique["prob"]

In [7]:
%%time
tirage=[]
for n in range(nbMillions*million):
    tirage.append(random.randrange(rangeTop))

CPU times: user 6min 35s, sys: 20.5 s, total: 6min 56s
Wall time: 7min 49s


In [8]:
lexique["tir1"]=0

In [9]:
%%time
def tirage2triage(tirage):    
    triage=sorted(tirage)
    freqTop=0
    indexMin=0
    tirs={}

    for num,tir in enumerate(triage[:]):
        if tir > freqTop:
            indexMin=lexique[lexique["freqcum"]>=tir][0:1].index.astype(int)[0]
            freqTop=lexique.ix[indexMin,'freqcum']
            tirs[indexMin]=0
        tirs[indexMin] += 1
        if num%500000==0:
            print num,
    print
    for indexNum in tirs:
        lexique.ix[indexNum,'tir1']+=tirs[indexNum]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [ ]:
portion=(nbMillions*million)//nTiles
print portion

200000000


In [ ]:
for nTile in range(nTiles):
    %time tirage2triage(tirage[nTile*portion:(nTile+1)*portion])
    print
    print "nTile",nTile
    print lexique[lexique["tir1"]!=0]["tir1"].sum()
    print lexique[lexique["tir1"]!=0]["tir1"].count()
    

In [ ]:
with open("/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+tiragePrefix+echantillon+'-Tirage.pkl', 'wb') as output:
    pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
print lexique[lexique["tir1"]!=0]["tir1"].sum()
print lexique[lexique["tir1"]!=0]["tir1"].count()